<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/PHI3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/microsoft/Phi-3-mini-128k-instruct

In [ ]:
!pip install colab-env --quiet
!pip install tiktoken -q
!pip install accelerate -q


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet

In [2]:
!nvidia-smi

Wed Apr 24 09:04:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   41C    P8              16W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Mounted at /content/gdrive
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

#model_id = "davidkim205/Rhea-72b-v0.5" # HF AVERAGE = 81.22 #1 APRIL 5TH, 2024

model_id = "microsoft/Phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True, do_sample=True)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [5]:
def generate_answer(prompt):
   outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.1,top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)
   #return outputs[0]['generated_text'][len(prompt):].strip()
   return outputs[0]['generated_text']


In [ ]:
response = generate_answer("What is the capital of canada?")

In [7]:
print(response)

What is the capital of canada?

### response: The capital of Canada is Ottawa. Ottawa is located in the eastern part of the southern province of Ontario, along the Ottawa River. It is the fourth largest city in Canada and serves as the political center of the country. Ottawa was chosen as the capital by Queen Victoria of the United Kingdom in 1857. The city is home to many national historic sites, including Parliament Hill, which houses the Canadian Parliament and the iconic Peace Tower. The presence of these institutions, along with its role as the seat of the federal government, makes Ottawa the capital of Canada.


In [8]:
print(f"Generated Answer:\n\n{response})")

Generated Answer:

What is the capital of canada?

### response: The capital of Canada is Ottawa. Ottawa is located in the eastern part of the southern province of Ontario, along the Ottawa River. It is the fourth largest city in Canada and serves as the political center of the country. Ottawa was chosen as the capital by Queen Victoria of the United Kingdom in 1857. The city is home to many national historic sites, including Parliament Hill, which houses the Canadian Parliament and the iconic Peace Tower. The presence of these institutions, along with its role as the seat of the federal government, makes Ottawa the capital of Canada.)


In [9]:
prompt="What is the capital of canada?"

In [10]:
messages = [
    {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."},
    {"role": "user", "content": prompt},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
output = pipe(messages, **generation_args)

In [12]:
print(output[0]['generated_text'])

 The capital of Canada is Ottawa. It is located in the eastern part of the province of Ontario, near the city of Montreal and the U.S. border. Ottawa is the fourth largest city in Canada and serves as the political center of the country, housing important institutions such as the Parliament Hill, the official residence of the Prime Minister, and the residence of the Governor General. Ottawa is known for its beautiful architecture, rich history, and vibrant cultural scene. It offers numerous museums, galleries, and festivals, as well as outdoor activities like hiking and canoeing in nearby natural areas. Additionally, Ottawa is a diverse and multicultural city, with a population that includes people from various ethnic backgrounds and cultures. It is also home to many universities and research institutions, contributing to its reputation as a hub for education and innovation. If you have any more questions or need further information, feel free to ask!


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

messages = [
    {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}




In [14]:
output = pipe(messages, **generation_args)

In [15]:
print(output[0]['generated_text'])

 To solve the equation 2x + 3 = 7, you need to isolate the variable x. Here are the steps:

1. Subtract 3 from both sides of the equation to get: 2x = 4.
2. Divide both sides of the equation by 2 to solve for x: x = 2.

So, the solution to the equation 2x + 3 = 7 is x = 2.
